In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import random
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras import activations

In [2]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3
...,...,...,...,...,...,...,...,...,...,...
196,91e8db1b91acc7bf903bf670461db3,f65c8906c07b627a760a0671de7144,26ba677f7ff11ac529f5f69c54cb0b,be6360babdb2e5910c5bc6f4f04f0d,705c23ed5ae1bc8824fc955b82094b,74733260a8a5143b2eb20c4a62940a,afed0e34e5af6958fef610a0ac5565,0a49f9b4576a63be41cb96908eba5e,baec94926106f54090568b653e84b4,8a2b5f96cc30660b28636fbe2664d5
197,c718bcdf6240802ba7123a7a631199,5e62a64d4f0d6ca3fce397267bf9f3,c5d40f1adeb446df77e65815da405d,bbf4b4a3db602ab9b88845114f7244,0f462e6bbd719e012902d8814d0e35,e5bca888fec7f8c0116db0002a7077,8a2483e11828332a9e2fc8659ae38f,bb59c82aeb5e44eb5735d5a1c016b8,8d3877a1b26b10642d125ec4fc4c9b,ae90166aa81ab2bf878dace61a5620
198,ee78a3aae1a69f2e4b82f26d41d594,11fe2dd4cccaf0567c878e19e1811a,aab18e84a97780bafb188fd33cb2a9,838cb9d91dd080147bcc18982bcdbb,972821d03656142f33707487f51b66,d6d45793f5b093e8bdf3d8ae4952b6,28731720ee3330b7107cec47168b8c,dad45a9302ad916f6901b86a744cdd,644bf6d0bdf6c9cddf71f5ab4cf7d4,4774caaf01df3d5d982e0cb77b1a13
199,b0b062dc25c4dada316183941b100c,80870ec50515107ba7034ab758cebe,d49c4a9b748b9c7abb9d98cdecebef,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf,67f32aff5488778cb4bbb6b21acc7d,3d558c3e37ccb70937b171aeb67bbb,97474799458a67e30a70f66407420b,0dbe493275a25285829b979316d566,ba54f9d576e93f2b0cf4622a92310f


In [3]:
movies_id = sessions_df.to_numpy()
movies_id = np.array(movies_id).flatten()
vocab = np.unique(movies_id)
vocab_len = len(vocab)

print(vocab)
print(vocab_len)
print(movies_id.shape)

['000b366b20d0f41187f6ae895e5d7a' '001db9af2dcc57d45d653aeb33171e'
 '00d3113bae64107671f55b51e183b9' '016254be26e2789ef3253e95ded1ea'
 '019fa155167f13b7be3444c46810ce' '01ab32976170c8eebe4054fd66cc39'
 '02817e267c3e407bc7b207f6ed9f34' '02a7f8e9b88ca2a74c6637332b0998'
 '02fbcf93a730c9c9d5e65aef41c67e' '046f93c91d0befd4625f00e07eb473'
 '05c92546aa14ea6148088b7c713be4' '06030fb2a64f0fc9df81ff76eea9d5'
 '06132809afbe1c781e1b9605452081' '061a59839837a123a916a70947f075'
 '06cd00fa7fce77cc7c1ad0a64d163e' '06d4e0812e40c50f93b3407878e744'
 '06f71f27fd9318f8b0f4f4dd69e522' '0713394356c9d9daf29edc39231352'
 '077374b7238b28b0a4aefa135f2f45' '07e873c5aac8ffaf97897fbd0dfbf6'
 '08712d35cf70496630e16b24dae005' '09bf18e7baee13e303f4b77aeaadbc'
 '0a091e4ffb26e3fc15d37939cd5682' '0a49f9b4576a63be41cb96908eba5e'
 '0b5aaa6d4b9941b2f4a31f9480d757' '0bdcec51b6924793d1fbd815daaa51'
 '0c691e3e6b52752ab6717b3729e326' '0dbe493275a25285829b979316d566'
 '0e0c51266d47f2c02625e16f61fa50' '0f462e6bbd719e012902d8814d0

In [4]:
def vocab_creater(arr):
    word2idx = {}
    idx2word = {}
    for i in range(len(arr)):
        word2idx[arr[i]] = i
        idx2word[i] = arr[i]

    return word2idx, idx2word, len(arr)

In [5]:
word2idx, idx2word, _ = vocab_creater(vocab)

In [6]:
print(_, word2idx)

366 {'000b366b20d0f41187f6ae895e5d7a': 0, '001db9af2dcc57d45d653aeb33171e': 1, '00d3113bae64107671f55b51e183b9': 2, '016254be26e2789ef3253e95ded1ea': 3, '019fa155167f13b7be3444c46810ce': 4, '01ab32976170c8eebe4054fd66cc39': 5, '02817e267c3e407bc7b207f6ed9f34': 6, '02a7f8e9b88ca2a74c6637332b0998': 7, '02fbcf93a730c9c9d5e65aef41c67e': 8, '046f93c91d0befd4625f00e07eb473': 9, '05c92546aa14ea6148088b7c713be4': 10, '06030fb2a64f0fc9df81ff76eea9d5': 11, '06132809afbe1c781e1b9605452081': 12, '061a59839837a123a916a70947f075': 13, '06cd00fa7fce77cc7c1ad0a64d163e': 14, '06d4e0812e40c50f93b3407878e744': 15, '06f71f27fd9318f8b0f4f4dd69e522': 16, '0713394356c9d9daf29edc39231352': 17, '077374b7238b28b0a4aefa135f2f45': 18, '07e873c5aac8ffaf97897fbd0dfbf6': 19, '08712d35cf70496630e16b24dae005': 20, '09bf18e7baee13e303f4b77aeaadbc': 21, '0a091e4ffb26e3fc15d37939cd5682': 22, '0a49f9b4576a63be41cb96908eba5e': 23, '0b5aaa6d4b9941b2f4a31f9480d757': 24, '0bdcec51b6924793d1fbd815daaa51': 25, '0c691e3e6b52752a

In [7]:
def toArray(smth):
    new = []
    
    for i in range(len(smth)):
        new.append(word2idx[smth[i]])
    
    return new


tokenized = toArray(movies_id)
print(tokenized)

[104, 60, 141, 332, 90, 8, 64, 166, 333, 108, 243, 63, 121, 291, 263, 189, 62, 345, 190, 350, 192, 304, 13, 19, 151, 312, 259, 299, 161, 181, 318, 9, 47, 76, 242, 201, 164, 66, 79, 354, 59, 285, 279, 132, 225, 317, 310, 289, 320, 340, 314, 61, 229, 218, 166, 108, 90, 332, 141, 8, 142, 101, 211, 170, 186, 10, 365, 331, 313, 41, 141, 332, 90, 8, 64, 166, 333, 108, 347, 335, 76, 9, 242, 201, 164, 66, 79, 354, 72, 57, 332, 141, 90, 8, 64, 166, 333, 108, 347, 335, 164, 201, 242, 76, 9, 66, 79, 238, 57, 72, 250, 116, 300, 165, 267, 224, 223, 244, 103, 55, 96, 20, 267, 165, 224, 223, 327, 119, 156, 244, 227, 194, 319, 70, 324, 157, 118, 177, 185, 220, 107, 150, 191, 328, 154, 364, 318, 188, 125, 222, 254, 212, 205, 297, 261, 269, 88, 145, 43, 315, 19, 304, 13, 179, 249, 129, 144, 232, 226, 82, 61, 314, 229, 218, 166, 108, 90, 332, 141, 8, 9, 318, 47, 76, 242, 201, 164, 66, 79, 354, 99, 162, 228, 27, 274, 104, 159, 60, 141, 332, 204, 358, 237, 252, 16, 74, 316, 209, 247, 342, 137, 128, 173, 15

In [8]:
dataset = tf.data.Dataset.from_tensor_slices(tokenized[:1400])

In [9]:
seq_length = 10
examples_per_epoch = len(tokenized) // (seq_length + 1)

In [10]:
sequences = dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(seq)

tf.Tensor([104  60 141 332  90   8  64 166 333 108 243], shape=(11,), dtype=int32)


In [11]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 10), (64, 10)), types: (tf.int32, tf.int32)>

In [12]:
for input_example, target_example in dataset.take(1):
    print("Input :", input_example)
    print("Target:", target_example)

Input : tf.Tensor(
[[328 154 364 318 188 125 222 254 212 205]
 [157 234 113 207 339 273 238   9  47  76]
 [141 332  90   8  64 229 314  61 218 166]
 [254 212 205 297 261 269  88 145  43  79]
 [233 116 250 300 165 267 224 223 244 103]
 [142 211 186 152  41 313 365 331 203 282]
 [313  41 331 275  10 170 211 101 186 101]
 [211 101 186 286 176 257 169  23 276 219]
 [ 51  47 102  99 162 228  27 274 104  21]
 [275  10 170 211 101 186 160  98 131 321]
 [173 322 137 155 128  14 214 363 123 142]
 [254 205 269 212  88 145  43 315 149 148]
 [ 90   8  64 166 333 302 306 105  95 334]
 [ 41 348 354 124  76   9 201 164 242  66]
 [212 205 297 261 269  88 145  71 239 153]
 [303 228 102 162  27 274 104 342 338 110]
 [205 212 254 338 110 247 342 175 202 270]
 [235 101 142 123 363 144 226 129 249 179]
 [  2  42 290 145  88 269  43 315 174 127]
 [238  57  72  90 141 332   8  64 166 333]
 [ 90 332 141   8 284 240 130  89  28  40]
 [222  53 104  60 141 332  90  24 297 208]
 [104  60 141 332  90   8  64 166 3

In [31]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation=activations.softmax)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        print(x.shape)

        if return_state:
            return x, states
        else:
            return x

In [32]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 1024

# Number of RNN units
rnn_units = 2048
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 366)
(64, 10, 366) # (batch_size, sequence_length, vocab_size)


In [26]:
tf.config.run_functions_eagerly(True)
model.summary()

Model: "my_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  374784    
_________________________________________________________________
gru_1 (GRU)                  multiple                  18886656  
_________________________________________________________________
dense_1 (Dense)              multiple                  749934    
Total params: 20,011,374
Trainable params: 20,011,374
Non-trainable params: 0
_________________________________________________________________


In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 10, 366)  # (batch_size, sequence_length, vocab_size)
Mean loss:         5.9032464


In [29]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [30]:
EPOCHS = 70
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/70
tf.Tensor(
[[[0.00276622 0.00277603 0.00275072 ... 0.00267672 0.00270258 0.00276568]
  [0.00275319 0.00277787 0.00272266 ... 0.00270172 0.00274138 0.00274187]
  [0.00270427 0.00275731 0.00276615 ... 0.00270101 0.00277864 0.00273871]
  ...
  [0.00269299 0.00270674 0.00276693 ... 0.00275482 0.00270996 0.00271176]
  [0.0026774  0.00272617 0.00273235 ... 0.00272099 0.00274168 0.00269975]
  [0.00269135 0.00274045 0.00275965 ... 0.00274453 0.00272681 0.00272525]]

 [[0.00278399 0.00274164 0.00278872 ... 0.00273338 0.00272414 0.00270864]
  [0.00275835 0.00273724 0.00275051 ... 0.00277529 0.00277038 0.00268972]
  [0.00272319 0.00274647 0.00271504 ... 0.00272471 0.00277661 0.00269007]
  ...
  [0.00273656 0.00271769 0.00277483 ... 0.00273235 0.00273815 0.00269828]
  [0.00274226 0.00274274 0.00277587 ... 0.00275983 0.00276571 0.00271626]
  [0.00268335 0.00273754 0.00276992 ... 0.00272333 0.00274635 0.00269654]]

 [[0.00268978 0.00275541 0.00274097 ... 0.00271435 0.00274401 0.00274367]


1/1 [==============================] - 1s 991ms/step - loss: 5.7706 - accuracy: 0.4719
Epoch 4/70
tf.Tensor(
[[[0.00275563 0.00274108 0.00269715 ... 0.00272959 0.0028122  0.00279803]
  [0.00272389 0.00270449 0.00269022 ... 0.00278823 0.0027551  0.00282451]
  [0.00278848 0.00262702 0.00264238 ... 0.00276852 0.00271227 0.00280476]
  ...
  [0.00274575 0.00260374 0.00262956 ... 0.00279834 0.0027845  0.00288202]
  [0.00269649 0.00259716 0.00269089 ... 0.00276073 0.00275358 0.00285891]
  [0.00270134 0.00262944 0.00265997 ... 0.00280542 0.00275005 0.00283862]]

 [[0.0027125  0.00265331 0.00274914 ... 0.00272916 0.00278679 0.00274146]
  [0.00268202 0.00262312 0.00272589 ... 0.00277425 0.00275526 0.00277718]
  [0.0026878  0.00263805 0.00271249 ... 0.00280903 0.00277622 0.00283315]
  ...
  [0.00269525 0.00264208 0.0026373  ... 0.00278891 0.00276624 0.00280229]
  [0.00268831 0.00265568 0.00267205 ... 0.00281971 0.00276409 0.00282004]
  [0.00272059 0.00260919 0.00266622 ... 0.00277449 0.00274708 0

1/1 [==============================] - 1s 979ms/step - loss: 5.3569 - accuracy: 0.2391
Epoch 7/70
tf.Tensor(
[[[2.64788792e-03 2.57079676e-03 2.66479026e-03 ... 2.80223694e-03
   2.81286775e-03 2.81168171e-03]
  [2.53848056e-03 2.46156077e-03 2.54729250e-03 ... 2.78711529e-03
   2.78502866e-03 2.84391991e-03]
  [2.39811279e-03 2.30114115e-03 2.40974734e-03 ... 2.93124351e-03
   2.80910823e-03 3.11461301e-03]
  ...
  [3.85611464e-04 2.87435745e-04 3.51868832e-04 ... 1.52692618e-03
   1.03068049e-03 2.86780088e-03]
  [5.12688093e-05 3.37917991e-05 4.62409807e-05 ... 3.99283454e-04
   2.17191715e-04 1.00446399e-03]
  [1.96007113e-06 1.05575987e-06 1.70292788e-06 ... 3.59827864e-05
   1.53275214e-05 1.38011907e-04]]

 [[2.62003508e-03 2.62744841e-03 2.67214538e-03 ... 2.73970678e-03
   2.76562502e-03 2.79520405e-03]
  [2.50164792e-03 2.51878821e-03 2.57297186e-03 ... 2.80324300e-03
   2.77372357e-03 2.88488390e-03]
  [2.41610641e-03 2.26673437e-03 2.42061703e-03 ... 2.85999058e-03
   2.830

1/1 [==============================] - 1s 1s/step - loss: 5.2081 - accuracy: 0.3531
Epoch 10/70
tf.Tensor(
[[[0.00263528 0.00259462 0.00246016 ... 0.0028824  0.00271084 0.00306383]
  [0.00258031 0.00250297 0.00238381 ... 0.00311859 0.0027346  0.00310531]
  [0.00249025 0.00225433 0.00223351 ... 0.00326622 0.00274905 0.00334131]
  ...
  [0.00101753 0.00091543 0.00108611 ... 0.00329541 0.00233637 0.00386434]
  [0.00088255 0.00070294 0.0008935  ... 0.00289553 0.0021132  0.00383987]
  [0.00086244 0.00060676 0.00075195 ... 0.00245924 0.00200583 0.00408245]]

 [[0.00259902 0.00247874 0.00253961 ... 0.00276949 0.00277246 0.00286229]
  [0.00249435 0.00239814 0.00233233 ... 0.00259809 0.00287706 0.00314528]
  [0.00238663 0.00227733 0.00216245 ... 0.00241921 0.00287851 0.00349676]
  ...
  [0.00218629 0.00200759 0.00195431 ... 0.00303013 0.00265307 0.00496741]
  [0.00221729 0.00196538 0.00194971 ... 0.00320188 0.00265848 0.00449036]
  [0.00214233 0.00189921 0.0018737  ... 0.00335542 0.00269527 0.0

1/1 [==============================] - 1s 1s/step - loss: 4.9868 - accuracy: 0.3797
Epoch 13/70
tf.Tensor(
[[[2.6925376e-03 2.7042481e-03 2.7027056e-03 ... 2.7399643e-03
   2.7566976e-03 2.7455038e-03]
  [2.6303623e-03 2.5747381e-03 2.7073182e-03 ... 2.7889127e-03
   2.7605654e-03 2.7419245e-03]
  [2.6073891e-03 2.5844122e-03 2.6417216e-03 ... 2.8317263e-03
   2.7509923e-03 2.7897430e-03]
  ...
  [2.3205986e-03 2.3602033e-03 2.3454924e-03 ... 3.2551624e-03
   2.8189765e-03 3.0048387e-03]
  [2.2827154e-03 2.3059323e-03 2.2593609e-03 ... 3.2573936e-03
   2.8251519e-03 3.1668590e-03]
  [2.2476520e-03 2.1952726e-03 2.1394000e-03 ... 3.4447168e-03
   2.8594811e-03 3.4495345e-03]]

 [[2.4941154e-03 2.4795455e-03 2.5938519e-03 ... 2.7731096e-03
   2.7729594e-03 2.9461519e-03]
  [2.3089759e-03 2.3718681e-03 2.4735800e-03 ... 2.8027291e-03
   2.7610394e-03 3.1608231e-03]
  [2.1692945e-03 2.2442222e-03 2.3176754e-03 ... 2.7440703e-03
   2.7676406e-03 3.4281739e-03]
  ...
  [8.3111727e-04 6.42480

...

1/1 [==============================] - 1s 1s/step - loss: 2.6390 - accuracy: 0.4875
Epoch 31/70
tf.Tensor(
[[[2.4686041e-03 2.2886305e-03 2.7467581e-03 ... 1.9913011e-03
   2.4558778e-03 2.2755268e-03]
  [8.5826463e-04 7.1798015e-04 1.0478284e-03 ... 8.4561791e-04
   1.5533466e-03 2.1634619e-03]
  [2.5682457e-04 2.0218095e-04 3.3670277e-04 ... 1.3912108e-04
   4.2295631e-04 1.3629071e-03]
  ...
  [1.5960912e-05 1.5237302e-05 1.2281640e-05 ... 2.5136262e-06
   1.5489348e-05 2.0084134e-03]
  [1.2236074e-05 1.3805044e-05 9.2693290e-06 ... 4.1577860e-06
   2.0658097e-05 3.9625494e-03]
  [5.4158313e-06 5.9985532e-06 3.7089087e-06 ... 7.1753170e-06
   2.3155559e-05 5.1321206e-03]]

 [[1.7819416e-03 1.8750553e-03 1.8797420e-03 ... 7.2440277e-03
   3.5032078e-03 4.1477997e-03]
  [6.1026250e-04 6.9335708e-04 6.5497181e-04 ... 9.5557114e-03
   4.3764296e-03 5.3635244e-03]
  [2.4330218e-05 2.9405499e-05 3.3272212e-05 ... 2.0950730e-03
   9.4351370e-04 2.0513770e-03]
  ...
  [3.6400289e-07 3.23003

1/1 [==============================] - 1s 966ms/step - loss: 2.2440 - accuracy: 0.5328
Epoch 34/70
tf.Tensor(
[[[5.52122947e-04 5.80880616e-04 5.01261442e-04 ... 2.73164921e-03
   2.83789146e-03 6.31625997e-03]
  [7.59674185e-06 9.41658618e-06 7.88892794e-06 ... 1.57245435e-04
   2.22416449e-04 1.24236022e-03]
  [5.13155456e-08 9.04861963e-08 6.74029863e-08 ... 4.25151529e-06
   3.92793163e-06 4.13752277e-05]
  ...
  [1.44874139e-06 8.65911773e-07 7.31420300e-07 ... 3.12210045e-06
   7.17521471e-06 3.43679894e-05]
  [1.53367330e-06 1.39966403e-06 8.61519709e-07 ... 5.39820076e-06
   1.23099335e-05 3.51383380e-04]
  [7.45722161e-07 1.06547100e-06 4.40906462e-07 ... 1.19244560e-05
   1.73255758e-05 1.97357708e-03]]

 [[1.40265829e-03 1.44351553e-03 1.74558396e-03 ... 1.35039631e-03
   8.31959827e-04 1.67254184e-03]
  [5.99150138e-04 4.72184474e-04 6.15933037e-04 ... 1.41261422e-04
   1.13913178e-04 1.88999649e-04]
  [5.65657720e-05 4.47857601e-05 4.69847837e-05 ... 3.38248492e-06
   6.45

1/1 [==============================] - 1s 1s/step - loss: 1.8534 - accuracy: 0.6000
Epoch 37/70
tf.Tensor(
[[[1.2116793e-03 1.2106708e-03 1.1582107e-03 ... 4.5057917e-03
   4.2257425e-03 4.0802667e-03]
  [7.2519545e-04 8.3521561e-04 6.9275755e-04 ... 1.5143581e-03
   2.5534390e-03 4.2405189e-03]
  [9.7821605e-05 1.1429918e-04 1.1992699e-04 ... 2.7939561e-04
   6.3643145e-04 7.2485260e-03]
  ...
  [7.5135979e-08 9.1272980e-08 5.0470568e-08 ... 2.0025495e-06
   4.4095918e-06 1.6761835e-03]
  [2.2803826e-08 4.0081929e-08 1.5651464e-08 ... 1.8728423e-06
   4.3505829e-06 1.6329326e-03]
  [3.5452528e-09 7.4022997e-09 2.8249698e-09 ... 1.1074327e-06
   1.8502230e-06 4.0363963e-04]]

 [[6.6069880e-04 6.6153420e-04 6.8104040e-04 ... 5.6346599e-03
   7.0654284e-03 1.2434223e-02]
  [1.1021080e-04 1.3084436e-04 1.2983491e-04 ... 1.3963359e-02
   8.3027231e-03 1.1182084e-02]
  [2.1174670e-05 2.3668325e-05 2.3420796e-05 ... 8.9311004e-03
   1.4683919e-02 1.6586918e-02]
  ...
  [4.3414259e-09 2.58527

1/1 [==============================] - 1s 980ms/step - loss: 1.5820 - accuracy: 0.6562
Epoch 40/70
tf.Tensor(
[[[1.5747157e-03 1.5990263e-03 1.5745336e-03 ... 3.7576901e-03
   3.7268791e-03 4.8948228e-03]
  [1.2069414e-03 1.3139755e-03 1.3738942e-03 ... 3.2164275e-03
   4.2276555e-03 6.2245862e-03]
  [3.7372447e-04 3.7928158e-04 4.3929889e-04 ... 3.3708685e-03
   6.8093431e-03 1.0253936e-02]
  ...
  [1.1702223e-07 6.8976249e-08 8.7184674e-08 ... 5.0108746e-08
   5.6160604e-07 8.2123658e-04]
  [6.5859666e-08 4.5903150e-08 3.8680216e-08 ... 3.1636059e-08
   3.1403204e-07 9.8689937e-04]
  [2.7561766e-08 3.7295408e-08 2.0140941e-08 ... 1.6667579e-08
   2.4094393e-07 2.8451202e-03]]

 [[4.8602291e-04 5.0598424e-04 4.7297595e-04 ... 1.2362956e-03
   1.9665915e-03 1.5751303e-03]
  [1.5163603e-05 1.6400174e-05 1.2327570e-05 ... 3.8671024e-05
   8.6053864e-05 4.6294674e-05]
  [1.9033637e-06 2.1442177e-06 1.0269307e-06 ... 2.3378307e-06
   5.6185818e-06 2.1865878e-06]
  ...
  [1.2465431e-07 3.93

KeyboardInterrupt: 

In [21]:
train = tf.data.Dataset.from_tensor_slices(tokenized[1024:])
sequences = train.batch(seq_length + 1, drop_remainder=True)
train = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

train = (
    train
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

/Users/nutochkina/PycharmProjects/diploma/venv/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


In [22]:
model.evaluate(train, batch_size=64, return_dict=True)

1/1 [==============================] - 0s 227ms/step - loss: 4.6762 - accuracy: 0.5016


{'loss': 4.676165580749512, 'accuracy': 0.5015624761581421}